In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from imutils import paths
import pandas as pd

%matplotlib inline


In [2]:

def trasformarImagens():
    global imagem
    global imagem_original

    #Colocando bordas na imagem
    imagem = cv2.copyMakeBorder(imagem, 50, 50, 50, 50, cv2.BORDER_REPLICATE)
    
    imagem_original = imagem

    #Borra bastante
    imagem = cv2.GaussianBlur(imagem,(75,75),0)

    #Limiar binary invertido + limiar de OTSU - o método de Otsu determina um valor de limite global ideal a partir do histograma da imagem
    limiar, imagem = cv2.threshold(imagem,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU )

    imagem = imagem*255
    imagem = imagem.astype(np.uint8)

    
    


In [3]:
def extrairDados():
    global imagem
    global imagem_original
    global nome
    global contornos

    aux = 1
    df = pd.DataFrame()
    for cnt in contornos:
       #area
        area = cv2.contourArea(cnt)
        
        #perimetro
        perimetro = cv2.arcLength(cnt,True)

        #compacidade
        compacidade = (perimetro**2)/area
        #print("Compacidade: ", compacidade)

        #circulatidade
        circularidade = (4*np.pi*area)/perimetro**2
        #print("circulatidade: ", circulatidade)

        #solidez
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        solidity = float(area)/hull_area
        #print("Solidez: ", solidity)

        #Cria eclipse envolvente 
        (xc,yc), (d1,d2), angle  = cv2.fitEllipse(cnt)

        #Pegas os pontos do eixo maior
        rmajor = max(d1,d2)/2
        if angle > 90:
            angle = angle - 90
        else:
            angle = angle + 90

        x1 = xc + math.cos(math.radians(angle))*rmajor
        y1 = yc + math.sin(math.radians(angle))*rmajor
        x2 = xc + math.cos(math.radians(angle+180))*rmajor
        y2 = yc + math.sin(math.radians(angle+180))*rmajor
        #calcula distancia
        eixo_maior = np.sqrt((x2-x1)**2 + (y2-y1)**2)

        #Pegas os pontos do eixo menor
        rminor = min(d1,d2)/2
        if angle > 90:
            angle = angle - 90
        else:
            angle = angle + 90

        x1 = xc + math.cos(math.radians(angle))*rminor
        y1 = yc + math.sin(math.radians(angle))*rminor
        x2 = xc + math.cos(math.radians(angle+180))*rminor
        y2 = yc + math.sin(math.radians(angle+180))*rminor
        #calcula distancia
        eixo_menor = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    
        #Razão do eixo menor pelo eixo maior
        razao_eixos = eixo_menor/eixo_maior
        #print("Razão Eixos: ", razao_eixos)

        #Proporção
        x,y,w,h = cv2.boundingRect(cnt)
        

        rect_area = w*h
        extent = float(area)/rect_area
        #print("Extent : ", extent)

        
        mask = np.zeros(imagem_original.shape,np.uint8)
        mask = cv2.drawContours(mask,[cnt],0,255,-1)
        #pixelpoints = np.transpose(np.nonzero(mask))
        

        mean_val = cv2.mean(imagem_original,mask = mask)
        auxiliar = mean_val[0]/255
        df = df.append({'Folha': aux, 'Compacidade': compacidade, 'Circularidade': circularidade, 'Solidez': solidity,
                        'Razão Eixos': razao_eixos, 'Extenção': extent, "Intencidade Média": auxiliar}, ignore_index=True)
        imagem_contornos = cv2.drawContours(cv2.cvtColor(imagem_original, cv2.COLOR_BAYER_BG2BGRA), [cnt], 0, (222,0,0), 3)

        cv2.imwrite(nome + "\contorno_" + str(aux) + ".bmp", imagem_contornos)
        #print("Intencidade media : ", razao_eixos)
        #print("")

        aux += 1
    df.to_excel(nome + "\contorno_" + str(aux-1) + 'saida.xlsx', index = False)


Main

In [4]:
imagemPath = list(paths.list_images("Imagens\Treinamento"))
numero = 1
global imagem
global nome
global contornos

if not os.path.exists("ImagensTratadas"):
    os.makedirs("ImagensTratadas")
    

for i in imagemPath:
    nomeAux = i.removeprefix("Imagens\Treinamento") 
    imagem = cv2.imread(i,0)#Transformar em tons de cinza

    """   kernel = np.ones((3,3),np.uint8)
    imagem = cv2.dilate(imagem, kernel,iterations = 3) """
    #print(i)
    
    trasformarImagens()
        
    contornos, void = cv2.findContours(imagem, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    nome = "ImagensTratadas\\" + nomeAux.removesuffix(".bmp") +"_" + str(len(contornos))
    if not os.path.exists("ImagensTratadas\\" + nomeAux.removesuffix(".bmp") +"_" + str(len(contornos))):
        os.makedirs("ImagensTratadas\\" + nomeAux.removesuffix(".bmp") +"_" + str(len(contornos)))

    extrairDados()       

    #print(i.replace(i, tipo +str(numero)+".jpg"))

    numero += 1

    #size = len(contornos)
    #print(size)

C:\Users\bianc\AppData\Local\Temp\ipykernel_3096\1509267739.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Folha': aux, 'Compacidade': compacidade, 'Circularidade': circularidade, 'Solidez': solidity,
C:\Users\bianc\AppData\Local\Temp\ipykernel_3096\1509267739.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Folha': aux, 'Compacidade': compacidade, 'Circularidade': circularidade, 'Solidez': solidity,
C:\Users\bianc\AppData\Local\Temp\ipykernel_3096\1509267739.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Folha': aux, 'Compacidade': compacidade, 'Circularidade': circularidade, 'Solidez': solidity,
C:\Users\bianc\AppData\Local\Temp\ipykernel_3096\1509267739.p